# Mars Web Scraping

In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

### Mars News

In [2]:
# requests call to page
marspage = requests.get("https://mars.nasa.gov/news")

In [3]:
# get page text
marspagetext = bs(marspage.text,'html.parser')
# marspagetext

In [4]:
# get article title and body
newstitle = marspagetext.find_all(class_='content_title')[0].a.text.strip()
newspara = marspagetext.find_all(class_='rollover_description_inner')[0].text.strip()
newstitle, newspara

('NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network',
 'Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.')

### Mars Picture

In [5]:
# start up splinter
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [6]:
# navigate site
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
browser.click_link_by_id('full_image')
browser.click_link_by_partial_text('more info')
html = browser.html
imagepagetext = bs(html, 'lxml')
browser.quit()

C:\Users\kosib\.conda\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:504: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [7]:
# extract image url
imageurl = 'https://www.jpl.nasa.gov' + imagepagetext.find(class_='main_image')['src'].strip()
imageurl

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18791_hires.jpg'

### Weather Twitter

In [8]:
# get twitter
marstwitter = requests.get("https://twitter.com/marswxreport?lang=en")
# get page text
marstwittertext = bs(marstwitter.text,'html.parser')
# marstwittertext

In [9]:
twitterweather = marstwittertext.find(class_="tweet-text").text.strip()
twitterweather

'InSight sol 439 (2020-02-20) low -94.7ºC (-138.4ºF) high -9.3ºC (15.2ºF)\nwinds from the SSE at 6.5 m/s (14.6 mph) gusting to 23.2 m/s (51.9 mph)\npressure at 6.30 hPapic.twitter.com/VRiv3fQH9p'